In [1]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import time
import math
import community as community_louvain
import networkx.algorithms.community as modularity
from networkx.algorithms.community.quality import modularity
from collections import defaultdict


G = nx.read_edgelist("C:/Users/gnssl/Downloads/facebook_combined.txt", 
                     create_using=nx.Graph(), nodetype=int)

partition_louvain = community_louvain.best_partition(G)
len_partition = len(partition_louvain)

population_size = 200
generations = 1000
crossover_prob = 1.0
mutation_prob = 0.01
local_optimization_frequency=5

min_partition_size = 5
max_partition_size = 15

def initialize_population():
    population = []
    for idx in range(population_size):
        
        partition = [random.randint(0, max_partition_size) for _ in range(len(G.nodes))]
        population.append(partition)
    return population

def compute_fitness(partition):
    communities = {key: value for key, value in zip(partition_louvain.keys(), partition)}
    new_dict = defaultdict(list)

    for key, value in communities.items():
        new_dict[value].append(key)

    result_dict = dict(new_dict)
    fitness = nx.community.modularity(G, result_dict.values())
    return fitness


def select_parents(population, fitness_values):
    parent1, parent2 = random.sample(population, 2)
    if compute_fitness(parent1) > compute_fitness(parent2):
        return parent1
    else:
        return parent2

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual, mutation_prob, max_partition_size):
    for i in range(len(individual)):
        if random.random() < mutation_prob:
            # Mutate the community assignment of the node to a random community
            individual[i] = random.randint(0, max_partition_size - 1)
    return individual

def partition_to_communities(partition):
    communities = {}
    for node, community_id in enumerate(partition):
        if community_id in communities:
            communities[community_id].add(node)
        else:
            communities[community_id] = {node}
    return list(communities.values())



def genetic_algorithm():
    population = initialize_population()
    max_fitness_values = []  # Store max fitness values for each generation

    for generation in range(generations):
        fitness_values = [compute_fitness(partition) for partition in population]
        max_fitness = max(fitness_values)
        max_fitness_values.append(max_fitness)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        print('Generation', generation , 'Max:', max_fitness)
        print('Generation', generation , 'Avg:', avg_fitness)

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parents(population, fitness_values)
            parent2 = select_parents(population, fitness_values)
            if random.random() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                
                child1 = mutate(child1, mutation_prob, max_partition_size)
                child2 = mutate(child2, mutation_prob, max_partition_size)
                
                min_fitness_index = min(range(len(fitness_values)), key=fitness_values.__getitem__)
                population[min_fitness_index] = child1
                second_min_fitness_index = min(range(len(fitness_values)), key=lambda x: fitness_values[x] if x != min_fitness_index else float('inf'))
                population[second_min_fitness_index] = child2
                
        
        if generation % local_optimization_frequency == 0:
            random_idx = random.randint(1,population_size//2)
            for idx in range(0, random_idx, population_size//random_idx):
                partition_louvain = community_louvain.best_partition(G)
                partition = [partition_louvain[node] for node in G.nodes]
                population[idx] = partition
       
    # Select the best partition from the final population
    best_partition = max(population, key=compute_fitness)
    return best_partition, max_fitness_values

start = time.time()
best_partition, max_fitness_values = genetic_algorithm()

best_communities = partition_to_communities(best_partition)
end = time.time()
print('time:', end - start)

print("Best Partition:", best_communities)

Generation 0 Max: 0.002425491123884548
Generation 0 Avg: -0.0004897956539304911
Generation 1 Max: 0.8349778014181175
Generation 1 Avg: 0.09142122408724745
Generation 2 Max: 0.8349778014181175
Generation 2 Avg: 0.0914404725499082
Generation 3 Max: 0.8349778014181175
Generation 3 Avg: 0.09145576598036444
Generation 4 Max: 0.8349778014181175
Generation 4 Avg: 0.09527402391427692
Generation 5 Max: 0.8349778014181175
Generation 5 Avg: 0.09529461493691917
Generation 6 Max: 0.8349778014181175
Generation 6 Avg: 0.09532003083438893
Generation 7 Max: 0.8349778014181175
Generation 7 Avg: 0.09533508132977798
Generation 8 Max: 0.8349778014181175
Generation 8 Avg: 0.09535055826295581
Generation 9 Max: 0.8349778014181175
Generation 9 Avg: 0.09537411024938124
Generation 10 Max: 0.8349778014181175
Generation 10 Avg: 0.10342412498838974
Generation 11 Max: 0.8349778014181175
Generation 11 Avg: 0.13265668446315274
Generation 12 Max: 0.8349778014181175
Generation 12 Avg: 0.1361785678362052
Generation 13 Ma

Generation 108 Max: 0.8358246351128347
Generation 108 Avg: 0.7804381114487177
Generation 109 Max: 0.8358246351128347
Generation 109 Avg: 0.7802007417144143
Generation 110 Max: 0.8358246351128347
Generation 110 Avg: 0.7808135416290378
Generation 111 Max: 0.8358246351128347
Generation 111 Avg: 0.7818372980217184
Generation 112 Max: 0.8358246351128347
Generation 112 Avg: 0.7832101951787597
Generation 113 Max: 0.8358246351128347
Generation 113 Avg: 0.7829222650291193
Generation 114 Max: 0.8358246351128347
Generation 114 Avg: 0.7832573687072808
Generation 115 Max: 0.8358246351128347
Generation 115 Avg: 0.7841761743575232
Generation 116 Max: 0.8358246351128347
Generation 116 Avg: 0.7853813462243165
Generation 117 Max: 0.8358246351128347
Generation 117 Avg: 0.7857690582745955
Generation 118 Max: 0.8358246351128347
Generation 118 Avg: 0.7869406744302941
Generation 119 Max: 0.8358246351128347
Generation 119 Avg: 0.7878889849659615
Generation 120 Max: 0.8358246351128347
Generation 120 Avg: 0.788

Generation 214 Max: 0.8358246351128347
Generation 214 Avg: 0.8122480891702277
Generation 215 Max: 0.8358246351128347
Generation 215 Avg: 0.8129764745846287
Generation 216 Max: 0.8358246351128347
Generation 216 Avg: 0.8130268149460207
Generation 217 Max: 0.8358246351128347
Generation 217 Avg: 0.8131579175781285
Generation 218 Max: 0.8358246351128347
Generation 218 Avg: 0.8135420717307799
Generation 219 Max: 0.8358246351128347
Generation 219 Avg: 0.812545469623167
Generation 220 Max: 0.8358246351128347
Generation 220 Avg: 0.8137860063343958
Generation 221 Max: 0.8358517125739899
Generation 221 Avg: 0.8138375104587479
Generation 222 Max: 0.8358517125739899
Generation 222 Avg: 0.8138475468105477
Generation 223 Max: 0.8358517125739899
Generation 223 Avg: 0.8131863958694755
Generation 224 Max: 0.8358517125739899
Generation 224 Avg: 0.8133576852613997
Generation 225 Max: 0.8358517125739899
Generation 225 Avg: 0.8139670560811251
Generation 226 Max: 0.8358517125739899
Generation 226 Avg: 0.8146

Generation 320 Max: 0.8357445765533564
Generation 320 Avg: 0.8182496462145109
Generation 321 Max: 0.8357946550432036
Generation 321 Avg: 0.8189036256526592
Generation 322 Max: 0.8357946550432036
Generation 322 Avg: 0.8181540863886254
Generation 323 Max: 0.8357946550432036
Generation 323 Avg: 0.8191869395328277
Generation 324 Max: 0.8357946550432036
Generation 324 Avg: 0.8185950262416073
Generation 325 Max: 0.8357946550432036
Generation 325 Avg: 0.8192599185820651
Generation 326 Max: 0.8357946550432036
Generation 326 Avg: 0.8176866836985416
Generation 327 Max: 0.8357946550432036
Generation 327 Avg: 0.8191199118049244
Generation 328 Max: 0.8357946550432036
Generation 328 Avg: 0.8186085283663712
Generation 329 Max: 0.8357946550432036
Generation 329 Avg: 0.8189282520515416
Generation 330 Max: 0.8357946550432036
Generation 330 Avg: 0.819339845801962
Generation 331 Max: 0.8357946550432036
Generation 331 Avg: 0.8193841083112183
Generation 332 Max: 0.8357946550432036
Generation 332 Avg: 0.8193

Generation 426 Max: 0.8358539153966056
Generation 426 Avg: 0.8198331901742896
Generation 427 Max: 0.8358539153966056
Generation 427 Avg: 0.8196424348938102
Generation 428 Max: 0.8358539153966056
Generation 428 Avg: 0.8204084253215801
Generation 429 Max: 0.8358539153966056
Generation 429 Avg: 0.8205816953553939
Generation 430 Max: 0.8358539153966056
Generation 430 Avg: 0.8207215551383275
Generation 431 Max: 0.8358539153966056
Generation 431 Avg: 0.8211111886107322
Generation 432 Max: 0.8358539153966056
Generation 432 Avg: 0.821094368105401
Generation 433 Max: 0.8358539153966056
Generation 433 Avg: 0.8210412112247363
Generation 434 Max: 0.8358539153966056
Generation 434 Avg: 0.8211610367026999
Generation 435 Max: 0.8358539153966056
Generation 435 Avg: 0.8208653217629076
Generation 436 Max: 0.8358539153966056
Generation 436 Avg: 0.8211637360881215
Generation 437 Max: 0.8358539153966056
Generation 437 Avg: 0.8212365889408442
Generation 438 Max: 0.8358539153966056
Generation 438 Avg: 0.8206

Generation 533 Max: 0.8355353739134573
Generation 533 Avg: 0.8216310555665632
Generation 534 Max: 0.8355353739134573
Generation 534 Avg: 0.8218701113422856
Generation 535 Max: 0.8355353739134573
Generation 535 Avg: 0.8225212460616406
Generation 536 Max: 0.8355353739134573
Generation 536 Avg: 0.8217753844706234
Generation 537 Max: 0.8355353739134573
Generation 537 Avg: 0.822043252911189
Generation 538 Max: 0.8355353739134573
Generation 538 Avg: 0.822465986356899
Generation 539 Max: 0.8355353739134573
Generation 539 Avg: 0.8223496101838752
Generation 540 Max: 0.8355353739134573
Generation 540 Avg: 0.8223242466030412
Generation 541 Max: 0.8355353739134573
Generation 541 Avg: 0.8224732941501194
Generation 542 Max: 0.8355353739134573
Generation 542 Avg: 0.8212997064723544
Generation 543 Max: 0.8355353739134573
Generation 543 Avg: 0.821426524614796
Generation 544 Max: 0.8355353739134573
Generation 544 Avg: 0.822172025767285
Generation 545 Max: 0.8355353739134573
Generation 545 Avg: 0.8217135

Generation 639 Max: 0.8357792944352957
Generation 639 Avg: 0.8223052088686642
Generation 640 Max: 0.8357792944352957
Generation 640 Avg: 0.822088523157041
Generation 641 Max: 0.8357792944352957
Generation 641 Avg: 0.8220056068592585
Generation 642 Max: 0.8357792944352957
Generation 642 Avg: 0.8230260130669522
Generation 643 Max: 0.8357792944352957
Generation 643 Avg: 0.8229494289892268
Generation 644 Max: 0.8357792944352957
Generation 644 Avg: 0.8221374380432037
Generation 645 Max: 0.8357792944352957
Generation 645 Avg: 0.821979546944931
Generation 646 Max: 0.8357792944352957
Generation 646 Avg: 0.822948461926721
Generation 647 Max: 0.8357792944352957
Generation 647 Avg: 0.8230868201423466
Generation 648 Max: 0.8357792944352957
Generation 648 Avg: 0.8217802636408313
Generation 649 Max: 0.8357792944352957
Generation 649 Avg: 0.8226538744873527
Generation 650 Max: 0.8357792944352957
Generation 650 Avg: 0.8217736565114149
Generation 651 Max: 0.8357792944352957
Generation 651 Avg: 0.823160

Generation 745 Max: 0.8349932717850226
Generation 745 Avg: 0.8229989756620507
Generation 746 Max: 0.8349932717850226
Generation 746 Avg: 0.8226229821826929
Generation 747 Max: 0.8349932717850226
Generation 747 Avg: 0.8228013960770555
Generation 748 Max: 0.8349932717850226
Generation 748 Avg: 0.8232326157673203
Generation 749 Max: 0.8349932717850226
Generation 749 Avg: 0.8235745067835655
Generation 750 Max: 0.8349932717850226
Generation 750 Avg: 0.8232451369183158
Generation 751 Max: 0.8349932717850226
Generation 751 Avg: 0.8233655831961231
Generation 752 Max: 0.8349932717850226
Generation 752 Avg: 0.8226167020869205
Generation 753 Max: 0.8349932717850226
Generation 753 Avg: 0.8231944076985864
Generation 754 Max: 0.8349932717850226
Generation 754 Avg: 0.8225526946202752
Generation 755 Max: 0.8349932717850226
Generation 755 Avg: 0.8226148827461214
Generation 756 Max: 0.8358363719397792
Generation 756 Avg: 0.8234721627283642
Generation 757 Max: 0.8358363719397792
Generation 757 Avg: 0.822

Generation 851 Max: 0.8358441302117978
Generation 851 Avg: 0.8238393703108285
Generation 852 Max: 0.8358441302117978
Generation 852 Avg: 0.823899922249539
Generation 853 Max: 0.8358441302117978
Generation 853 Avg: 0.8236678857755635
Generation 854 Max: 0.8358441302117978
Generation 854 Avg: 0.824041699468208
Generation 855 Max: 0.8358441302117978
Generation 855 Avg: 0.8238230928032951
Generation 856 Max: 0.8358441302117978
Generation 856 Avg: 0.8235754057421869
Generation 857 Max: 0.8358441302117978
Generation 857 Avg: 0.8236572580042012
Generation 858 Max: 0.8358441302117978
Generation 858 Avg: 0.8241305489607129
Generation 859 Max: 0.8358441302117978
Generation 859 Avg: 0.8231280575504161
Generation 860 Max: 0.8358441302117978
Generation 860 Avg: 0.8230619905454571
Generation 861 Max: 0.8358441302117978
Generation 861 Avg: 0.8233217735220475
Generation 862 Max: 0.8358441302117978
Generation 862 Avg: 0.8237954904626047
Generation 863 Max: 0.8358441302117978
Generation 863 Avg: 0.82408

Generation 957 Max: 0.8358441302117978
Generation 957 Avg: 0.8233868436796083
Generation 958 Max: 0.8358441302117978
Generation 958 Avg: 0.8243673363108224
Generation 959 Max: 0.8358441302117978
Generation 959 Avg: 0.8239577567279172
Generation 960 Max: 0.8358441302117978
Generation 960 Avg: 0.8241330950862233
Generation 961 Max: 0.8358441302117978
Generation 961 Avg: 0.8242670387018507
Generation 962 Max: 0.8358441302117978
Generation 962 Avg: 0.8233045684992862
Generation 963 Max: 0.8358441302117978
Generation 963 Avg: 0.8242495073926928
Generation 964 Max: 0.8358441302117978
Generation 964 Avg: 0.8244328098840775
Generation 965 Max: 0.8358441302117978
Generation 965 Avg: 0.8242437857553869
Generation 966 Max: 0.8358441302117978
Generation 966 Avg: 0.8233399620888261
Generation 967 Max: 0.8358441302117978
Generation 967 Avg: 0.8238895666387447
Generation 968 Max: 0.8358441302117978
Generation 968 Avg: 0.8234658868495466
Generation 969 Max: 0.8358441302117978
Generation 969 Avg: 0.824